# Image Colourisation using Autoencoders

## Imports
Importing all the necessary libraries required to run the following code for image colourization.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualising and plotting results

import os

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

import torchvision.transforms as T
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm.notebook import tqdm
from skimage.color import rgb2lab, lab2rgb, rgb2gray

## Loading the Data

Defining the directory for the dataset to be used.

In [ ]:
DATA_DIR = '../input/imagenet'

Loading images contained in the data directory into a avriable called dataset using ImageFolder, and applying two transforms to all the images:
1. Resizing the non-uniformly sized images into 256x256 images.
2. Converting them into tensors.

In [ ]:
dataset = ImageFolder(DATA_DIR, transform=T.Compose([T.Resize((256, 256)),T.ToTensor()]))

In [ ]:
len(dataset)

Creating the training and validation sets. A random seed value is set to have the same training and validation datasets each time the notebook is run, and the two datasets are split using the random_split function from the pytorch libraries.

In [ ]:
random_seed = 42
torch.manual_seed(random_seed)

val_size = 1000
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

The batch size is set to be 128.

In [ ]:
batch_size = 128

Loading the training and validation datasets into the CPU using DataLoader.

In [ ]:
train_loader = DataLoader(train_ds, batch_size = batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_loader = DataLoader(val_ds, batch_size = batch_size, num_workers = 4, pin_memory = True)

Use CUDA if available else fall back to CPU for training.

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

Define a function to load data into the device assigned above.

In [ ]:
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking = True)

Define a custom DataLoader to load data onto the available device.

In [ ]:
class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device

    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch, self.device)
  
    def __len__(self):
        return len(self.dl)

Wrapping the training and validation dataloaders in our custom and efficient `DeviceDataLoader`.

In [ ]:
train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)

## Defining the Models
The images are originally using the RGB colormodel. But for the purpose of image colourization, they have to be converted to Lab. The CIELAB is a colorspace which has 3 channels: L, a, and b. The function defined below converts images from RGB to CIELAB, and then splits the L and ab channels into the variables X and Y respectively. The range of ab channels is from -128 to 127, hence, the ab channels are normalized by dividing it by 128. The function returns finally returns X and Y loaded into cuda.

In [ ]:
def generate_l_ab(images): 
    lab = rgb2lab(images.permute(0, 2, 3, 1).cpu().numpy())
    X = lab[:,:,:,0]
    X = X.reshape(X.shape+(1,))
    Y = lab[:,:,:,1:] / 128
    return to_device(torch.tensor(X, dtype = torch.float).permute(0, 3, 1, 2), device),to_device(torch.tensor(Y, dtype = torch.float).permute(0, 3, 1, 2), device)

The Base Model class is defined as containing three functions. 
1. The *training_batch* function takes the batch of 128 images as input, generates the values of L and ab channels using the *generate_l_ab* function, gets the predicted ab channels using the forward function of the model, and calculates the MSE Loss between the actual and predicted ab channels.
2. The *validation_batch* functions performs the same task as the *training_batch* function, except for the images in the validation dataset, which is evident from the function names.
3. The function *validation_end_epoch* returns the average loss on the validation dataset.

In [ ]:
class BaseModel(nn.Module):
    def training_batch(self, batch):
        images, _ = batch
        X, Y = generate_l_ab(images)
        outputs = self.forward(X)
        loss = F.mse_loss(outputs, Y)
        return loss

    def validation_batch(self, batch):
        images, _ = batch
        X, Y = generate_l_ab(images)
        outputs = self.forward(X)
        loss = F.mse_loss(outputs, Y)
        return {'val_loss' : loss.item()}

    def validation_end_epoch(self, outputs):
        epoch_loss = sum([x['val_loss'] for x in outputs]) / len(outputs)
        return {'epoch_loss' : epoch_loss}

A helper function is defined to get the appropriate padding in order to keep the size of the output image same as the input image during convolution.

In [ ]:
def get_padding(kernel_size: int, stride: int = 1, dilation: int = 1, **_) -> int:
    padding = ((stride - 1) + dilation * (kernel_size - 1)) // 2
    return padding

The `Encoder_Decoder_v1` class is an extension of the Base Model and it contains the code for the version 1 architecture of the model.

In [ ]:
class Encoder_Decoder_v1(BaseModel):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size = 3, stride = 2, padding = get_padding(3, 2)),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size = 3, padding = get_padding(3)),
            nn.ReLU(),
            nn.BatchNorm2d(128),
        
            nn.Conv2d(128, 128, kernel_size = 3, stride = 2, padding = get_padding(3, 2)),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 256, kernel_size = 3, padding = get_padding(3)),
            nn.ReLU(),
            nn.BatchNorm2d(256),
        
            nn.Conv2d(256, 256, kernel_size = 3, stride = 2, padding = get_padding(3, 2)),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 512, kernel_size = 3, padding = get_padding(3)),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            
            nn.Conv2d(512, 512, kernel_size = 3, padding = get_padding(3)),
            nn.ReLU(),
            nn.BatchNorm2d(512),
            nn.Conv2d(512, 256, kernel_size = 3, padding = get_padding(3)),
            nn.ReLU(),
            nn.BatchNorm2d(256),
        
            nn.Conv2d(256, 128, kernel_size = 3, padding = get_padding(3)),
            nn.Upsample(size = (64,64)),
            nn.Conv2d(128, 64, kernel_size = 3, padding = get_padding(3)),
            nn.Upsample(size = (128,128)),
            nn.Conv2d(64, 32, kernel_size = 3, padding = get_padding(3)),
            nn.Conv2d(32, 16, kernel_size = 3, padding = get_padding(3)),
            nn.Conv2d(16, 2, kernel_size = 3, padding = get_padding(3)),
            nn.Tanh(),
            nn.Upsample(size = (256,256))
    )

    def forward(self, images):
        return self.network(images)     
    

The `Encoder_Decoder_v2` class is an extension of the Base Model and it contains the code for the version 2 architecture of the model which uses ResNet18 for the Encoder part.

In [ ]:
class Encoder_Decoder_v2(Base_Model):
    def __init__(self, input_size=128):
        super(Encoder_Decoder, self).__init__()
        MIDLEVEL_FEATURE_SIZE = 128
        
        
        # Encoder Part
        resnet = models.resnet18(num_classes=365) 
        resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
        self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6]) # Extracting the first
                                                                            # 6 convolutional layers
        
        # Decoder Part
        self.upsample = nn.Sequential(     
          nn.Conv2d(MIDLEVEL_FEATURE_SIZE, 128, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(128),
          nn.ReLU(),
          nn.Upsample(scale_factor=2),
          nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.Upsample(scale_factor=2),
          nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.Conv2d(32, 2, kernel_size=3, stride=1, padding=1),
          nn.Upsample(scale_factor=2)
        )
        
    def forward(self, input):

        # Pass input through ResNet-gray to extract features
        midlevel_features = self.midlevel_resnet(input)

        # Upsample to get colors
        output = self.upsample(midlevel_features)
        return output

The model is defined and loaded to cuda.

In [ ]:
model = Encoder_Decoder_v1()
to_device(model, device)

## Training the Model
The *validate* and *fit* functions are defined to keep track of the loss and train the model.

In [ ]:
@torch.no_grad()
def validate(model, val_loader):
    model.eval()
    outputs = [model.validation_batch(batch) for batch in val_loader]
    return model.validation_end_epoch(outputs)

def fit(model, epochs, learning_rate, train_loader, val_loader, optimization_func = torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    optimizer = optimization_func(model.parameters(), learning_rate)
    for epoch in range(epochs):
        train_losses = []
        model.train()
        for batch in tqdm(train_loader):
            loss = model.training_batch(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = validate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        history.append(result)
        print('Epoch: {}, Train loss: {:.4f}, Validation loss: {:.4f}'.format(epoch, result['train_loss'], result['epoch_loss']))
    return history

Initializing the loss.

In [ ]:
#history = [validate(model, val_loader)]
#history

Training.

In [ ]:
#history += fit(model, 5, 0.001, train_loader, val_loader, torch.optim.Adam)

Saving the model to be able to continue training the model from the same point.

In [ ]:
torch.save(model.state_dict(), 'test.pth')

Loading the pth file to continue training from the same parameters onwards.

In [ ]:
def load_checkpoint(filepath): 
    model = Encoder_Decoder_v1()
    model.load_state_dict(torch.load(filepath))
    
    return model

In [ ]:
model = load_checkpoint('../input/landscape-test/Landscapes.pth')
to_device(model, device)

## Testing the Model
Combining the L channel and the predicted ab channels and converting it to RGB to obtain the final colour image.

In [ ]:
def to_rgb(grayscale_input, ab_output):
    color_image = torch.cat((grayscale_input, ab_output), 0).numpy() # combine channels
    print(color_image.shape)
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1]
    color_image[:, :, 1:3] = (color_image[:, :, 1:3]) * 128
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    return color_image

Given a black and white image with 3 channels(R==G==B), converting it to Lab and giving the L channel as input to predict the ab channels and finally, obtaining the predicted coloured version of the image.

In [ ]:
def prediction(img):
    a = rgb2lab(img.permute(1, 2, 0))
    a = torch.tensor(a[:,:,0]).type(torch.FloatTensor)
    a = a.unsqueeze(0)
    a = a.unsqueeze(0)
    xb = to_device(a, device)
    ab_img = model(xb)
    xb = xb.squeeze(0)
    ab_img = ab_img.squeeze(0)
    return to_rgb(xb.detach().cpu(), ab_img.detach().cpu())

Testing the model on different black and white images and plotting the result.

In [ ]:
import glob
from PIL import Image

images = glob.glob("../input/grayscale-landscape/download (7).jpg")

In [ ]:
img = images[0]
image = Image.open(img)
trans = T.Compose([T.Resize((256, 256)),T.ToTensor()])
img = trans(image)

In [ ]:
f, arr = plt.subplots(1, 2, sharey=True)
arr[0].imshow(img.permute(1, 2, 0))
arr[1].imshow(prediction(img))